# Conexión al laboratorio SQL

In [1]:
!pip install sqlalchemy
!pip install mysql-connector-python

In [2]:
from sqlalchemy import create_engine
import sqlalchemy
import mysql.connector as mysql
import pandas as pd

# Creamos la conexión

In [3]:
# Parámetros de conexión
user     = 'root'
password = ''
# db = 
sqlServer = 'localhost' # debería ser IP o DNS (Domain Name System) del servidor de la base de datos

# Creamos string de conexión
connStr = "mysql+mysqlconnector://" + user + ":" + password + "@" + sqlServer # +  "/"  + db

# print de string de conexión
print (connStr)

# Conectamos con el servidor
conn = create_engine(connStr).connect() 

mysql+mysqlconnector://root:@localhost


# Creamos nuestra BD

In [4]:
# Crea una BD con nombre NUCLIO
sSQL = "CREATE DATABASE ENTREGABLE"
conn.execute(sSQL)

# De otra manera o si ya lo tenemos creado:
# conn.execute("DROP DATABASE nombre") # Para eliminar la base de datos si ya existe
# conn.execute("CREATE DATABASE nombre") # Par crear la base de datos

DatabaseError: (mysql.connector.errors.DatabaseError) 1007 (HY000): Can't create database 'ENTREGABLE'; database exists
[SQL: CREATE DATABASE ENTREGABLE]
(Background on this error at: https://sqlalche.me/e/14/4xp6)

In [5]:
# Comando para ver todas las bases de datos disponibles en el servidor
dbs = conn.execute("SHOW DATABASES").fetchall()
dbs

[('ENTREGABLE',),
 ('information_schema',),
 ('mysql',),
 ('NUCLIO',),
 ('performance_schema',),
 ('sys',)]

In [5]:
# Especificamos que queremos usar la base de datos NUCLIO
conn.execute("USE ENTREGABLE")

# Carga de datos

In [6]:
alojamiento = pd.read_excel('alojamiento.xlsx')
alojamiento.to_sql('alojamiento', conn)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1050 (42S01): Table 'alojamiento' already exists
[SQL: 
CREATE TABLE alojamiento (
	`index` BIGINT, 
	`CODIGO_ALOJAMIENTO` TEXT, 
	`NOMBRE` TEXT, 
	`CANT_BANYOS` BIGINT, 
	`CANT_HAB` BIGINT, 
	`SALON` TEXT, 
	`TERRAZA` TEXT, 
	`AIREC_ACOND` TEXT, 
	`PISCINA` TEXT, 
	`SUPERFICIE` BIGINT
)

]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [7]:
precio = pd.read_excel('precio.xlsx')
precio.to_sql('precio', conn)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1050 (42S01): Table 'precio' already exists
[SQL: 
CREATE TABLE precio (
	`index` BIGINT, 
	`CODIGO_ALOJAMIENTO` TEXT, 
	`PRECIO_ALQUILER` BIGINT, 
	`RESERVA` TEXT, 
	`PORCENTAJE_RESERVA` BIGINT
)

]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [8]:
puntuacion = pd.read_excel('puntuacion.xlsx')
puntuacion.to_sql('puntuacion', conn)


ProgrammingError: (mysql.connector.errors.ProgrammingError) 1050 (42S01): Table 'puntuacion' already exists
[SQL: 
CREATE TABLE puntuacion (
	`index` BIGINT, 
	`CODIGO_ALOJAMIENTO` TEXT, 
	`PUNTOS` FLOAT(53), 
	`AGENCIA` TEXT, 
	`PUNTOS_AGENCIA` FLOAT(53)
)

]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [9]:
ubicacion = pd.read_excel('ubicacion.xlsx')
ubicacion.to_sql('ubicacion', conn)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1050 (42S01): Table 'ubicacion' already exists
[SQL: 
CREATE TABLE ubicacion (
	`index` BIGINT, 
	`CODIGO_ALOJAMIENTO` TEXT, 
	`PAIS` TEXT, 
	`CIUDAD` TEXT, 
	`DIST_METRO` FLOAT(53), 
	`DIST_CENTRO` FLOAT(53)
)

]
(Background on this error at: https://sqlalche.me/e/14/f405)

# PRIMERA PARTE

## PASO 1

## Crear la query de cada uno de las exigencias del cliente. Se debe crear una query para cada punto. (7 queries en total)

## El cliente exige terraza y aire acondicionado, pero no quiere piscina.

In [10]:
sSQL = """
SELECT * FROM ALOJAMIENTO
WHERE TERRAZA = 'SI' 
AND AIREC_ACOND = 'SI' 
AND PISCINA = 'NO';
"""

pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,CANT_HAB,SALON,TERRAZA,AIREC_ACOND,PISCINA,SUPERFICIE
0,3,ALOJ004,Piso Centro Histórico,2,3,SI,SI,SI,NO,90
1,9,ALOJ010,Ático del Sol,3,3,SI,SI,SI,NO,88
2,14,ALOJ015,Dúplex Las Rocas,3,4,SI,SI,SI,NO,120
3,17,ALOJ018,Piso Parque Central,2,4,SI,SI,SI,NO,100


## No se alojará en estudios que no tengan salón ni en propiedades con solo un baño

In [11]:
sSQL = """
SELECT * FROM ALOJAMIENTO
WHERE SALON = 'SI' 
AND CANT_BANYOS > 1;
"""
pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,CANT_HAB,SALON,TERRAZA,AIREC_ACOND,PISCINA,SUPERFICIE
0,0,ALOJ001,Villa Sol del Mar,2,3,SI,SI,SI,SI,120
1,3,ALOJ004,Piso Centro Histórico,2,3,SI,SI,SI,NO,90
2,5,ALOJ006,Chalet Vista al Lago,3,4,SI,SI,SI,SI,150
3,6,ALOJ007,Dúplex Jardines del Sur,2,3,SI,SI,SI,SI,130
4,8,ALOJ009,Casa Rústica Montaña,2,2,SI,SI,NO,NO,85
5,9,ALOJ010,Ático del Sol,3,3,SI,SI,SI,NO,88
6,11,ALOJ012,Villa Palmeras,2,3,SI,SI,SI,SI,110
7,13,ALOJ014,Casa del Mar,3,4,SI,SI,SI,SI,160
8,14,ALOJ015,Dúplex Las Rocas,3,4,SI,SI,SI,NO,120
9,16,ALOJ017,Chalet Jardín Secreto,3,5,SI,SI,SI,SI,180


## La propiedad debe tener como mínimo 80m2

In [12]:
sSQL = """
SELECT * FROM ALOJAMIENTO
WHERE SUPERFICIE >= 80;
"""

pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,CANT_HAB,SALON,TERRAZA,AIREC_ACOND,PISCINA,SUPERFICIE
0,0,ALOJ001,Villa Sol del Mar,2,3,SI,SI,SI,SI,120
1,3,ALOJ004,Piso Centro Histórico,2,3,SI,SI,SI,NO,90
2,5,ALOJ006,Chalet Vista al Lago,3,4,SI,SI,SI,SI,150
3,6,ALOJ007,Dúplex Jardines del Sur,2,3,SI,SI,SI,SI,130
4,8,ALOJ009,Casa Rústica Montaña,2,2,SI,SI,NO,NO,85
5,9,ALOJ010,Ático del Sol,3,3,SI,SI,SI,NO,88
6,11,ALOJ012,Villa Palmeras,2,3,SI,SI,SI,SI,110
7,12,ALOJ013,Apartamento Moderno,1,2,SI,NO,SI,NO,80
8,13,ALOJ014,Casa del Mar,3,4,SI,SI,SI,SI,160
9,14,ALOJ015,Dúplex Las Rocas,3,4,SI,SI,SI,NO,120


## No quiere propiedades a menos de 1 km del metro ni a menos de 2 km del centro

In [13]:
sSQL = """
SELECT * FROM UBICACION
WHERE DIST_METRO >= 1 
AND DIST_CENTRO >= 2;
"""

pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,PAIS,CIUDAD,DIST_METRO,DIST_CENTRO
0,0,ALOJ001,Portugal,Liboa,1.0,2.0
1,3,ALOJ004,Francia,Marsella,3.0,4.0
2,4,ALOJ005,Portugal,Oporto,4.0,3.0
3,5,ALOJ006,Francia,Paris,2.0,2.0
4,6,ALOJ007,Francia,Toulouse,1.0,3.0
5,7,ALOJ008,España,Barcelona,2.0,3.0
6,8,ALOJ009,España,Madrid,2.0,4.0
7,9,ALOJ010,España,Bilbao,2.0,5.0
8,14,ALOJ015,España,Málaga,2.0,4.0
9,15,ALOJ016,Italia,Roma,2.0,2.0


## Precio entre 1.500€ y 2.000€ por noche

In [14]:
sSQL = """
SELECT * FROM PRECIO
WHERE PRECIO_ALQUILER 
BETWEEN 1500 AND 2000;
"""

pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,PRECIO_ALQUILER,RESERVA,PORCENTAJE_RESERVA
0,0,ALOJ001,1800,SI,100
1,3,ALOJ004,1600,SI,25
2,7,ALOJ008,1500,SI,25
3,8,ALOJ009,1600,SI,100
4,9,ALOJ010,1750,SI,25
5,10,ALOJ011,1825,NO,0
6,11,ALOJ012,1750,NO,0
7,12,ALOJ013,1652,NO,0
8,13,ALOJ014,1941,NO,0
9,14,ALOJ015,1800,SI,25


## El porcentaje de reserva debe ser exactamente 25%

In [15]:
sSQL = """
SELECT * FROM PRECIO
WHERE PORCENTAJE_RESERVA = 25;
"""

pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,PRECIO_ALQUILER,RESERVA,PORCENTAJE_RESERVA
0,1,ALOJ002,780,SI,25
1,3,ALOJ004,1600,SI,25
2,7,ALOJ008,1500,SI,25
3,9,ALOJ010,1750,SI,25
4,14,ALOJ015,1800,SI,25
5,17,ALOJ018,1900,SI,25


## Confianza mayor a 4.5 y puntuación de agencia mayor a 4

In [16]:
sSQL = """
SELECT * FROM PUNTUACION
WHERE PUNTOS > 4.5 
AND PUNTOS_AGENCIA > 4;
"""

pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,PUNTOS,AGENCIA,PUNTOS_AGENCIA
0,0,ALOJ001,5.0,GlobalHome,4.4
1,3,ALOJ004,4.6,CaribeRent,4.6
2,6,ALOJ007,4.9,SevillaRooms,4.5
3,7,ALOJ008,4.8,ChileVive,4.3
4,8,ALOJ009,4.7,MexicoRent,4.8
5,9,ALOJ010,4.8,UYAlquila,4.9
6,13,ALOJ014,4.7,BogotaLodge,4.5
7,14,ALOJ015,4.9,EasyStay Valencia,4.7
8,16,ALOJ017,4.9,CuscoRenta,4.4
9,17,ALOJ018,4.7,MendozaRooms,4.7


## PASO 2

# Crear en una única query la combinación de los filtros anteriores para hallar las propiedades que finalmente cumplen con todas las exigencias del cliente.

In [17]:
sSQL = """ 
SELECT A.CODIGO_ALOJAMIENTO, A.NOMBRE
FROM ALOJAMIENTO A
JOIN UBICACION UB ON A.CODIGO_ALOJAMIENTO = UB.CODIGO_ALOJAMIENTO
JOIN PRECIO PR ON A.CODIGO_ALOJAMIENTO = PR.CODIGO_ALOJAMIENTO
JOIN PUNTUACION PT ON A.CODIGO_ALOJAMIENTO = PT.CODIGO_ALOJAMIENTO
WHERE A.TERRAZA = 'SI'
  AND A.AIREC_ACOND = 'SI'
  AND A.PISCINA = 'NO'
  AND A.SALON = 'SI'
  AND A.CANT_BANYOS > 1
  AND A.SUPERFICIE >= 80
  AND UB.DIST_METRO >= 1
  AND UB.DIST_CENTRO >= 2
  AND PR.PRECIO_ALQUILER BETWEEN 1500 AND 2000
  AND PR.PORCENTAJE_RESERVA = 25
  AND PT.PUNTOS > 4.5
  AND PT.PUNTOS_AGENCIA > 4;
  """

pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE
0,ALOJ004,Piso Centro Histórico
1,ALOJ010,Ático del Sol
2,ALOJ015,Dúplex Las Rocas
3,ALOJ018,Piso Parque Central


## PASO 3

## Como habéis visto, tenemos 4 propiedades que cumplen con todas las exigencias de nuestro cliente, ahora bien, desde tu punto de vista, haciendo un análisis de los datos del cliente y de todas sus exigencias ¿Cuál de estos 4 alojamientos le recomendarías y por qué?

In [ ]:
# Desde mi punto de vista y analizando el resto de datos de los 4 alojamientos que cumplen requisitos, yo elegiría Dúplex las Rocas, ya 
# que es el que tiene más metros cuadrados, más habitaciones y más baños, además de que es el que tiene una ubicación en la playa, y al ser
# vacaciones de verano es un plus. Y la puntuación es suficientemente alta.

# SEGUNDA PARTE

## Responde las siguientes preguntas (con su respectiva query)

In [18]:
#  ¿Cuál es la suma total de metros cuadrados de los alojamientos que tienen piscina?

sSQL = """
SELECT SUM(SUPERFICIE) AS TOTAL_SUPERFICIE
FROM ALOJAMIENTO
WHERE PISCINA = 'SI';
"""

pd.read_sql(sSQL, conn)

,TOTAL_SUPERFICIE
0,850.0


In [19]:
# ¿Cuál es el alojamiento con mayor superficie?

sSQL = """
SELECT * FROM ALOJAMIENTO
ORDER BY SUPERFICIE DESC
LIMIT 1;
"""

pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,CANT_HAB,SALON,TERRAZA,AIREC_ACOND,PISCINA,SUPERFICIE
0,16,ALOJ017,Chalet Jardín Secreto,3,5,SI,SI,SI,SI,180


In [20]:
# ¿Cuál es el alojamiento con menor superficie?

sSQL = """
SELECT * FROM ALOJAMIENTO
ORDER BY SUPERFICIE ASC
LIMIT 1;
"""
pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,CANT_HAB,SALON,TERRAZA,AIREC_ACOND,PISCINA,SUPERFICIE
0,15,ALOJ016,Estudio Urbano,1,1,NO,NO,SI,NO,35


In [22]:
# ¿Cuántos alojamientos tienen salón?

sSQL = """
SELECT COUNT(*) AS CON_SALON
FROM ALOJAMIENTO
WHERE SALON = 'SI';
"""

pd.read_sql(sSQL, conn)

,CON_SALON
0,16


In [24]:
# ¿Cuántos alojamientos tienen terraza?

sSQL = """
SELECT COUNT(*) AS CON_TERRAZA
FROM ALOJAMIENTO
WHERE TERRAZA = 'SI';
"""

pd.read_sql(sSQL, conn)

,CON_TERRAZA
0,14


In [ ]:
# ¿Cuántos alojamientos hay por cada país según nuestra tabla de UBICACIÓN?

sSQL = """
SELECT PAIS, COUNT(*) AS TOTAL
FROM UBICACION
GROUP BY PAIS;
"""

pd.read_sql(sSQL, conn)

,PAIS,TOTAL
0,Portugal,4
1,España,8
2,Francia,4
3,Italia,4


In [27]:
# ¿Cuál es el promedio de precios de alquiler de todos los alojamientos según la tabla PRECIO?

sSQL = """
SELECT AVG(PRECIO_ALQUILER) AS PROMEDIO_ALQUILERES
FROM PRECIO;
"""

pd.read_sql(sSQL, conn)


,PROMEDIO_ALQUILERES
0,1568.4


In [28]:
# ¿Cuáles son las 3 agencias que tienen la mayor puntuación de agencia (campo a usar: PUNTOS_AGENCIA) según la tabla PUNTUACION?

sSQL = """
SELECT AGENCIA, PUNTOS_AGENCIA
FROM PUNTUACION
ORDER BY PUNTOS_AGENCIA DESC
LIMIT 3;
"""

pd.read_sql(sSQL, conn)

,AGENCIA,PUNTOS_AGENCIA
0,MalagaTurismo,4.9
1,VivaStay,4.9
2,UYAlquila,4.9


In [29]:
# Muestra los alojamientos (código y nombre) que: Contengan la palabra ‘Piso’ en su nombre o que tengan piscina y aire acondicionado, 
# pero no pasen los 150 metros de superficie o que tengan más de un baño

sSQL = """
SELECT CODIGO_ALOJAMIENTO, NOMBRE
FROM ALOJAMIENTO
WHERE NOMBRE LIKE '%Piso%'
   OR (PISCINA = 'SI' AND AIREC_ACOND = 'SI' AND SUPERFICIE <= 150)
   OR CANT_BANYOS > 1;"""

pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE
0,ALOJ001,Villa Sol del Mar
1,ALOJ004,Piso Centro Histórico
2,ALOJ006,Chalet Vista al Lago
3,ALOJ007,Dúplex Jardines del Sur
4,ALOJ009,Casa Rústica Montaña
5,ALOJ010,Ático del Sol
6,ALOJ012,Villa Palmeras
7,ALOJ014,Casa del Mar
8,ALOJ015,Dúplex Las Rocas
9,ALOJ017,Chalet Jardín Secreto


In [ ]:
# Muestra una consulta que muestre el código de alojamiento, el nombre, el precio y
# una columna donde veamos el precio del alquiler con el símbolo del euro al final de 
# cada importe, esta nueva columna se debe llama PRECIO_MODIF

sSQL = """
SELECT A.CODIGO_ALOJAMIENTO, A.NOMBRE, PR.PRECIO_ALQUILER,
       CONCAT(PR.PRECIO_ALQUILER, '€') AS PRECIO_MODIF
FROM ALOJAMIENTO A
JOIN PRECIO PR ON A.CODIGO_ALOJAMIENTO = PR.CODIGO_ALOJAMIENTO;
"""

pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,PRECIO_ALQUILER,PRECIO_MODIF
0,ALOJ001,Villa Sol del Mar,1800,1800€
1,ALOJ002,Apartamento Las Palmeras,780,780€
2,ALOJ003,Cabaña El Refugio,800,800€
3,ALOJ004,Piso Centro Histórico,1600,1600€
4,ALOJ005,Estudio Playa Blanca,900,900€
5,ALOJ006,Chalet Vista al Lago,1200,1200€
6,ALOJ007,Dúplex Jardines del Sur,1300,1300€
7,ALOJ008,Loft Urbano,1500,1500€
8,ALOJ009,Casa Rústica Montaña,1600,1600€
9,ALOJ010,Ático del Sol,1750,1750€


In [39]:
# Muestra una columna sobre la tabla PRECIO que se llame CATEGORIA_PRECIO y 
# que indique la palabra ‘Bajo’ si el precio está entre 780 y 1000, ‘Medio’ si el precio 
# está entre 1000 y 1700 y ‘Alto’ si es mayor a 1700

sSQL = """
SELECT CODIGO_ALOJAMIENTO, PRECIO_ALQUILER,
       CASE
         WHEN PRECIO_ALQUILER BETWEEN 780 AND 1000 THEN 'Bajo'
         WHEN PRECIO_ALQUILER BETWEEN 1001 AND 1700 THEN 'Medio'
         WHEN PRECIO_ALQUILER > 1700 THEN 'Alto'
       END AS CATEGORIA_PRECIO
FROM PRECIO;
"""

pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,PRECIO_ALQUILER,CATEGORIA_PRECIO
0,ALOJ001,1800,Alto
1,ALOJ002,780,Bajo
2,ALOJ003,800,Bajo
3,ALOJ004,1600,Medio
4,ALOJ005,900,Bajo
5,ALOJ006,1200,Medio
6,ALOJ007,1300,Medio
7,ALOJ008,1500,Medio
8,ALOJ009,1600,Medio
9,ALOJ010,1750,Alto
